In [9]:
import cv2
import numpy as np
import logging
import os
from collections import deque



def add_transparent_overlay(frame, boxes, color=(0, 0, 255), alpha=0.3):
    """
    Disegna rettangoli rossi trasparenti sulle aree di movimento.
    """
    overlay = frame.copy()
    for (x, y, w, h) in boxes:
        cv2.rectangle(overlay, (x, y), (x + w, y + h), color, -1)
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

def temporal_smoothing_flow(
    video_path, 
    output_dir, 
    flow_threshold=0.5,       # Soglia di magnitudo per considerare un pixel in movimento
    alpha_fraction=0.2,       # Percentuale di frame su N in cui il pixel deve essere acceso
    window_size=30,           # N: lunghezza finestra temporale
    morph_kernel=3,           # Dimensione kernel morfologico ridotto
    save_name="flow_smoothing_output.mp4",
    mask_save_name="movement_mask.mp4",  # Nome del video maschera binaria
    margin=10,                # Padding in pixel
    scale_factor=0.5,         # Riduzione di scala per il calcolo dell'Optical Flow
    skip_frames=0             # Numero di frame da saltare (0 = nessuno)
):
    """
    Rileva il movimento usando Farneback Optical Flow e produce:
      - Un video con overlay (rettangoli rossi) = save_name
      - Un video con maschera binaria (0/255) = mask_save_name
    """
     
    cap = cv2.VideoCapture(video_path)
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # VideoWriter per video con overlay
    fourcc_overlay = cv2.VideoWriter_fourcc(*'avc1')
    output_path = os.path.join(output_dir, save_name)
    out_overlay = cv2.VideoWriter(output_path, fourcc_overlay, fps, (width, height))

    # VideoWriter per la maschera binaria (isColor=False)
    fourcc_mask = cv2.VideoWriter_fourcc(*'avc1')
    mask_output_path = os.path.join(output_dir, mask_save_name)
    out_mask = cv2.VideoWriter(mask_output_path, fourcc_mask, fps, (width, height), isColor=False)

    ret, first_frame = cap.read()


    # Converto il primo frame in scala di grigi e riduco per Optical Flow
    prev_gray_full = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    prev_gray = cv2.resize(prev_gray_full, (0, 0), fx=scale_factor, fy=scale_factor)

    # Deque per maschere negli ultimi N frame
    mask_queue = deque(maxlen=window_size)
    frame_count = 1

    # Kernel morfologico
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_kernel, morph_kernel))

    while True:
        ret, frame = cap.read()
        if not ret:
            break


        gray_full = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray_full, (0, 0), fx=scale_factor, fy=scale_factor)

        # Optical Flow
        flow = cv2.calcOpticalFlowFarneback(
            prev_gray, gray, None,
            0.5,    # Scale
            2,      # Levels
            9,      # Winsize
            2,      # Iterations
            5,      # Poly_n
            1.2,    # Poly_sigma
            0       # Flags
        )

        # Magnitudo e Angolo
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1], angleInDegrees=False)
        mask_current_resized = (mag > flow_threshold).astype(np.uint8) * 255
        # Risali alla risoluzione originale
        mask_current = cv2.resize(mask_current_resized, (width, height), interpolation=cv2.INTER_NEAREST)

        # Aggiungi la maschera alla coda
        mask_queue.append(mask_current)

        # Costruisci la maschera "media" con smoothing temporale
        if frame_count <= window_size:
            cumulative_mask = np.sum(np.array(mask_queue), axis=0)
        else:
            cumulative_mask = np.sum(np.array(mask_queue), axis=0)

        mask_smoothed = (cumulative_mask >= (alpha_fraction * len(mask_queue) * 255)).astype(np.uint8) * 255

        # Operazioni morfologiche
        mask_smoothed = cv2.morphologyEx(mask_smoothed, cv2.MORPH_CLOSE, kernel)
        mask_smoothed = cv2.morphologyEx(mask_smoothed, cv2.MORPH_OPEN, kernel)

        # Trova contorni e crea bounding box
        contours, _ = cv2.findContours(mask_smoothed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        boxes = []
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            x_padded = max(0, x - margin)
            y_padded = max(0, y - margin)
            w_padded = min(w + 2*margin, width - x_padded)
            h_padded = min(h + 2*margin, height - y_padded)

            if w_padded > 5 and h_padded > 5:
                boxes.append((x_padded, y_padded, w_padded, h_padded))

        
        # Disegno overlay
        if boxes:
            add_transparent_overlay(frame, boxes, (0, 0, 255), alpha=0.3)


        # Crea la maschera binaria con rettangoli pieni
        mask_rect = np.zeros((height, width), dtype=np.uint8)
        for (x, y, w, h) in boxes:
            cv2.rectangle(mask_rect, (x, y), (x + w, y + h), 255, -1)  # -1 riempie il rettangolo

        # Scrivi la maschera binaria
        out_mask.write(mask_rect)

        # Opzionale: visualizza il frame con overlay
        #cv2.imshow("Temporal Smoothing Flow", frame)

        # Aggiorna
        prev_gray = gray.copy()
        frame_count += 1



    cap.release()
    out_overlay.release()
    out_mask.release()
    cv2.destroyAllWindows()
    logging.info(f"Video overlay salvato in: {output_path}")
    logging.info(f"Video maschera salvato in: {mask_output_path}")

def main():
    video_path = "../Dataset/input/test2.mp4"
    output_dir = "../Dataset/output/"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    temporal_smoothing_flow(
        video_path=video_path,
        output_dir=output_dir,
        flow_threshold=0.5,
        alpha_fraction=0.2,
        window_size=9,
        morph_kernel=3,
        save_name="flow_smoothing_output.mp4",
        mask_save_name="movement_mask.mp4",
        margin=20,
        scale_factor=0.5,
        skip_frames=0
    )

if __name__ == "__main__":
    main()

In [12]:
import cv2
import numpy as np
import logging
import os
import subprocess
from numba import jit

def setup_logging(output_dir):
    log_file = os.path.join(output_dir, "compression.log")
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.FileHandler(log_file),  # Salva log su file
            logging.StreamHandler()        # Mostra log su console
        ]
    )
    logging.info(f"Logging configurato. Log salvati in: {log_file}")


def process_blocks(mask, channels, QTY_aggressive, blockSize):
    """
    Ottimizza la quantizzazione dei blocchi statici con Numba per migliorare la velocità.
    """
    for i in range(0, mask.shape[0], blockSize):
        for j in range(0, mask.shape[1], blockSize):
            if mask[i:i+blockSize, j:j+blockSize].mean() == 0:  # Zona senza movimento
                for c in range(3):  # Per ogni canale (Y, Cr, Cb)
                    block = channels[c][i:i+blockSize, j:j+blockSize]
                    if block.shape == (blockSize, blockSize):  # Evita bordi incompleti
                        dct_block = cv2.dct(np.float32(block) - 128)
                        quantized_block = np.round(dct_block / QTY_aggressive) * QTY_aggressive
                        idct_block = cv2.idct(quantized_block) + 128
                        channels[c][i:i+blockSize, j:j+blockSize] = np.clip(idct_block, 0, 255).astype(np.uint8)
    return channels

def quantize_frame(frame_bgr, mask, blockSize=8, QTY_aggressive=None, scale_factor=0.5):
    """
    Applica la quantizzazione aggressiva alle aree statiche con riduzione della risoluzione.
    """
    if QTY_aggressive is None:
        QTY_aggressive = np.full((blockSize, blockSize), 100, dtype=np.float32)

    # Converti il frame in YCrCb e separa i canali
    frame_ycrcb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2YCR_CB)
    channels = cv2.split(frame_ycrcb)

    # Ridimensiona la maschera per calcoli più rapidi
    small_mask = cv2.resize(mask, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_NEAREST)
    small_channels = [cv2.resize(c, (0, 0), fx=scale_factor, fy=scale_factor) for c in channels]

    # Processa i blocchi statici sulla risoluzione ridotta
    small_channels = process_blocks(small_mask, small_channels, QTY_aggressive, blockSize)

    # Ridimensiona i canali elaborati alla risoluzione originale
    channels = [cv2.resize(c, (mask.shape[1], mask.shape[0])) for c in small_channels]

    # Ricombina i canali e riconverti in BGR
    return cv2.cvtColor(cv2.merge(channels), cv2.COLOR_YCR_CB2BGR)

def compress_frame_with_motion_mask(frame_bgr, mask, blockSize=8):
    """
    Wrapper per applicare la quantizzazione alle aree statiche.
    """
    return quantize_frame(frame_bgr, mask, blockSize)

def compress_with_ffmpeg_and_trim(input_path, output_path):
    """
    Applica una compressione HEVC con ffmpeg e ottimizza riducendo le dimensioni.
    """
    logging.info(f"Compressione HEVC e ottimizzazione avviata per: {input_path}")
    ffmpeg_command = [
        "ffmpeg",
        "-i", input_path,
        "-c:v", "libx265",  # Codec HEVC per alta compressione
        "-crf", "28",       # Compressione più aggressiva
        "-preset", "ultrafast",  # Ottimizzazione per velocità
        "-y",               # Sovrascrive l'output se esistente
        output_path
    ]
    try:
        subprocess.run(ffmpeg_command, check=True)
        logging.info(f"Compressione completata. Video ottimizzato salvato in: {output_path}")
    except subprocess.CalledProcessError as e:
        logging.error(f"Errore durante la compressione con ffmpeg: {e}")

def main_compress_with_motion(input_video, motionMask_video, output_video="COMPRESSION_quantized_static.mp4"):
    """
    Rimuove informazioni dalle aree statiche applicando quantizzazione e ottimizza la compressione.
    """
    cap_input = cv2.VideoCapture(input_video)
    cap_mask = cv2.VideoCapture(motionMask_video)

    if not cap_input.isOpened():
        logging.error(f"Impossibile aprire il video originale: {input_video}")
        return
    if not cap_mask.isOpened():
        logging.error(f"Impossibile aprire il video maschera: {motionMask_video}")
        return

    fps = cap_input.get(cv2.CAP_PROP_FPS)
    width = int(cap_input.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap_input.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # VideoWriter per il video di output
    fourcc = cv2.VideoWriter_fourcc(*'avc1')
    output_path = os.path.join(os.path.dirname(motionMask_video), output_video)
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0

    while True:
        ret_in, frame_in = cap_input.read()
        ret_mask, frame_mask = cap_mask.read()
        if not ret_in or not ret_mask:
            break

        frame_count += 1

        # Verifica dimensioni maschera e frame
        if frame_in.shape[:2] != frame_mask.shape[:2]:
            logging.error("Dimensioni di frame e maschera non corrispondono!")
            break

        # Quantizza le aree statiche
        processed_frame = compress_frame_with_motion_mask(
            frame_in, 
            frame_mask,
            blockSize=16
        )

        # Scrivi il frame nel video di output
        out.write(processed_frame)

    cap_input.release()
    cap_mask.release()
    out.release()
    cv2.destroyAllWindows()

    # Esegui compressione con ffmpeg e ottimizza
    optimized_output = output_path.replace(".mp4", "_optimized.mp4")
    compress_with_ffmpeg_and_trim(output_path, optimized_output)

def main():
    video_originale = "../Dataset/input/test2.mp4"
    output_dir = "../Dataset/output/"
    mask_video_path = os.path.join(output_dir, "movement_mask.mp4")  # Generato dal Blocco 1
    video_compresso = "COMPRESSION_quantized_static.mp4"

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    setup_logging(output_dir)

    main_compress_with_motion(
        input_video=video_originale,
        motionMask_video=mask_video_path,
        output_video=video_compresso
    )

if __name__ == "__main__":
    main()